# Advent of Code 2018

Hey there! For this year of Advent of Code, I'm going to be trying to journal my progress through the month's problems using a Python notebook, inspired by the similar notebooks of [Peter Norvig](https://github.com/norvig/pytudes). I've considered doing other things this year, like perhaps using the Advent of Code as an excuse to learn a new language, but last year I only completed the first 8, so I figured I should focus on first trying to complete all of this year's problems before getting ahead of myself. (Plus, the easier challenges make for good interview problem practice, so it fits in with my current goals).

Since I'm very comfortable with Python, I'm going to try to be competitive with timing (as the time to write code is the main bottleneck typically - not the program execution), so I will try to finish on the leaderboard. But this is my first time using an Python notebook, so there's a chance this will slow me down. Better hope for the best!

## Day 0: Imports and utilities

Following the inspiration of Peter Norvig, here I will try to put some code and imports that I think could be useful. For now I'll just include libraries and functions I've used a decent amount.

In [7]:
import bisect
import math
import random
import re
import string

from collections import Counter, defaultdict
from heapq import heappush, heappop
from itertools import accumulate, combinations, cycle, takewhile

# source: https://stackoverflow.com/a/30558049
def unique_permutations(elements):
    if len(elements) == 1:
        yield (elements[0],)
    else:
        unique_elements = set(elements)
        for first_element in unique_elements:
            remaining_elements = list(elements)
            remaining_elements.remove(first_element)
            for sub_permutation in unique_permutations(remaining_elements):
                yield (first_element,) + sub_permutation

def flatten(lst):
    return [elem for sublst in lst for elem in sublst]

## [Day 1](https://adventofcode.com/2018/day/1): Chronal Calibration

This problem was very straight forward. Getting a tight time is just a matter of having your environment setup, and fortunately I don't think using jupyter really slowed me down much, as I already had prepared some starter code in advance for reading in input.

Unfortunately, I took 1:48 to solve the first star, but to get in the top 100 I would have needed 1:32; and I took 6:32 on the second star, but to get in the top 100 I would have needed 5:28. Besides reading the problem as a bottleneck, during the second problem I was planning to loop over indices but ended up looping over values, so I ended up with an incorrect answer costing me a minute. Also, my code definitely looks pretty sloppy...

In [4]:
# first star

with open('input/input1.txt') as f:
    X = list(map(int, f.read().split()))

print(sum(X))

# second star

freqs = set()
curr = 0
while True:
    found = False
    for x in X:
        curr += x
        if curr in freqs:
            found = True
            print(curr)
            break
        freqs.add(curr)
    if found:
        break

556
448


Here's how I would refactor the second star with a few extra minutes to spare. The interesting thing about this kind of problem is that it seems simple enough that it should be doable with a Python one-or-two-liner, but I don't have an exact intuition for it (though I'm sure someone on Reddit has done it). That said, I think for most people this type of problem feels easier to solve this with iterative methods than functional programming.

In [5]:
# second star

freqs = set()
for x in accumulate(cycle(X)):
    if x in freqs:
        print(x)
        break
    freqs.add(x)

448


## [Day 2](https://adventofcode.com/2018/day/2): Inventory Management System

Unfortunately, I didn't have this time to do this challenge at midnight, so I can't really comment on speed for these questions, though they were only slightly more involved than Day 1. The first star is very easily solvable in Python using the Counter library function from the collections module - it's useful in quite a number of coding interview puzzles, in my experience.

The second star involves a bit more work, as it seems to necessitate comparing all pairs of strings against one another to decide which differs exactly by one character. Both parts, calculating the number of differences, and deriving the string in common, can be simplified as Python one-liners without adding much unnecessary overhead. Writing Python one-liners tend to just feel satisfying to derive and look clean to look at, even though they don't always reflect how a person came up with the line.

It's worth noting how out of habit (from websites like HackerRank and LeetCode), I tend to code defensively, doing things like iterating only through the minimum of the lengths of the two strings, even though all strings in the given input are the same length.

In [6]:
# first star

with open('input/input2.txt') as f:
    X = f.read().split()

twos, threes = 0, 0
for x in X:
    count = Counter(x)
    if 2 in count.values():
        twos += 1
    if 3 in count.values():
        threes += 1
print(twos * threes)

# second star

def num_differences(s1, s2):
    return sum([s1[i] != s2[i] for i in range(min(len(s1), len(s2)))])

def in_common(s1, s2):
    return ''.join([s1[i] for i in range(min(len(s1), len(s2))) if s1[i] == s2[i]])

for i in range(len(X)):
    for j in range(i + 1, len(X)):
        if num_differences(X[i], X[j]) == 1:
            print(in_common(X[i], X[j]))

7192
mbruvapghxlzycbhmfqjonsie


## [Day 3](https://adventofcode.com/2018/day/3): No Matter How You Slice It

That was satisfying for sure! I ended up solving this puzzle using a simple brute force approach, storing all of the positions in a grid and then performing the necessary operations to update the grid with appropriate ownership of the squares. For the second star, my first solution made the answer relatively easy to retrieve, since it just required me to iterate over in the same fashion and re-check the ownership of various regions.

Unfortunately, the main part I got stuck on was something I should probably be fluent with: input parsing. I knew how to use string.split() to separate the lines, but the complex format of the output made regex a clear candidate for the problem. That said, I haven't necessarily used regex a lot, so I was originally looking up ways to separate strings by multiple separators in Python, before deciding that just gathering all of the sequences of digits would be more efficient.

The other slight slowdown was that at the very end I remembered I had to flatten the list (or at least flattening would make for the cleanest solution), and I hadn't written a function for it in advance. Once these issues were figured out though, most of the coding was smooth sailing.

In [7]:
# first star

with open('input/input3.txt') as f:
    X = f.read().split('\n')[0:-1]  # remove the last line, which is empty

grid = [[None] * 1000 for _ in range(1000)]
for x in X:
    a, b, c, d, e = map(int, re.findall('\d+', x))
    for row in range(c, c + e):
        for col in range(b, b + d):
            if grid[row][col] is None:
                grid[row][col] = a
            else:
                grid[row][col] = 'X'

print(sum([f == 'X' for f in flatten(grid)]))

# second star

for x in X:
    a, b, c, d, e = map(int, re.findall('\d+', x))
    safe = True
    for row in range(c, c + e):
        for col in range(b, b + d):
            if grid[row][col] == 'X':
                safe = False
                break
        if not safe:
            break
    if safe:
        print(a)
        break

113966
235


## [Day 4](https://adventofcode.com/2018/day/4): Repose Record

This update is quite delayed, but as I became preoccupied with final exams, I had to take a temporary hiatus on Advent of Code. But now that I am on holiday break, there is more time to complete these challenges (though I won't be competing for leaderboard positions). Anyway, onward to the puzzle!

As with Day 3, I was slowed down a bit by first figuring out what the most efficient way to parse the data was. I used regular expressions again this time, but instead of splitting the string, I just used re.match while specifying groups in my pattern (using parentheses) so I could extract individual groups into tuples. I'm not sure if this is necessarily the most efficient method, but it's fairly readable and understandable, so I think it works fine. Since the time stamps of each string are fixed in size, it now occurs to me that I could probably have just picked slices out of the strings for the times, but I would still have to search for the guard IDs on the "begins shift" records.

The rest of the calculation itself was fairly simple. Once I extracted the data into tuples (ordered by month, day, hour, minute), sorting in Python automatically places all of the records in the right order. Then I stored individual data for guards using a dictionary, associating an array of 60 integers for each guard. Once I determined the time interval during which a guard was sleeping, I simply incremented the corresponding entries of the array. It's possible this could be handled more efficiently somehow (perhaps with some variant of range queries?) but for a list of size 60, we can sleep safely at night. After we have the sleeping data for each guard, calculating the most-slept minutes (and the total time they spent sleeping) was easy.

In [8]:
# first star

with open('input/input4.txt') as f:
    lines = f.readlines()

records = []
for line in lines:
    m = re.match(r"\[\d\d\d\d-(\d\d)-(\d\d) (\d\d):(\d\d)\] (.*)", line)
    records.append((int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4)), m.group(5)))

# generate guard data
records = sorted(records)
guard_data = dict()  # dictionary mapping guard ID's (ints) to length-60 arrays of sleep frequencies
curr_guard = -1
sleeping = -1
for record in records:
    m = re.search(r"\d+", record[4])
    if m:  # "Begin shift" record
        curr_guard = int(m.group(0))
        if curr_guard not in guard_data:
            guard_data[curr_guard] = [0] * 60
    elif record[4] == 'falls asleep':  # "falls asleep" record
        sleeping = record[3]
    elif record[4] == 'wakes up':  # "wakes up" record
        for i in range(sleeping, record[3]):
            guard_data[curr_guard][i] += 1
    else:
        raise Exception('could not read data: {}'.format(record))
        
# process guard data
best_id = ''
best_total = 0
best_minute = 0
for guard in guard_data:
    total = sum(guard_data[guard])
    if total > best_total:
        best_id = guard
        best_total = total
        best_minute = guard_data[guard].index(max(guard_data[guard]))

print(best_id * best_minute)

# second star

best_id = ''
best_freq = 0
best_minute = 0
for guard in guard_data:
    freq = max(guard_data[guard])
    if freq > best_freq:
        best_id = guard
        best_freq = freq
        best_minute = guard_data[guard].index(freq)

print(best_id * best_minute)

39698
14920


## [Day 5](https://adventofcode.com/2018/day/5) : Alchemical Reduction

For the first star, my initial instinct was to try and somehow convert the string into a data structure which allowed constant time intermediate deletions, like a doubly-linked list. But as it happens, doubly-linked lists aren't natively available in Python, and creating a datastructure for this didn't seem like it would be very efficient (unless I was using a lower level language like C/C++/Rust perhaps).

Fortunately, I realized I could accomplish the task easier if I just used a stack to continuously add elements to the "result" string, just checking if anything reacts with the top element in the stack. When one imagines the long chemical chain reacting in a vacuum, the first image that comes to my mind is a long line which slowly collapses inwards - but when you view it just from one side, the iterative approach clearly doesn't miss out on any reactions. This result can probably be shown easily using some form of proof by induction or contradiction.

For the second star, I reused the same approach to simply iterate through the newly reduced string with a stack, but additionally checking if the character I am currently reading is a fixed target letter (e.g. 'a'), in which case I ignore it and move to the next character. I repeat this for every possible fixed letter, and keep track of what the lowest string length I obtained overall was.

In [5]:
# first star

with open('input/input5.txt') as f:
    X = f.readline().strip()

stack = []
for char in X:
    if not stack:  # stack is empty
        stack.append(char)
    else:
        e1 = stack[-1]
        e2 = char
        if e1.lower() == e2.lower() and \
                ((e1 == e1.lower() and e2 == e2.upper()) or \
                 (e1 == e1.upper() and e2 == e2.lower())):
            stack.pop()
        else:
            stack.append(char)

print(len(stack))

# second star

min_length = len(stack)
for letter in string.ascii_lowercase:
    new_stack = []
    for char in stack:
        if char == letter or char == letter.upper():
            continue
        elif not new_stack:
            new_stack.append(char)
        else:
            e1 = new_stack[-1]
            e2 = char
            if e1.lower() == e2.lower() and \
                    ((e1 == e1.lower() and e2 == e2.upper()) or \
                     (e1 == e1.upper() and e2 == e2.lower())):
                new_stack.pop()
            else:
                new_stack.append(char)
    min_length = min(min_length, len(new_stack))

print(min_length)

10564
6336


## [Day 6](https://adventofcode.com/2018/day/6): Chronal Coordinates

This problem certainly off the bat seemed like a large difficulty spike compared to the previous puzzles. The problem was additionally challenging since I originally went down some incorrect paths, after intuitively thinking about the problem initially in terms of euclidaen distance, even though it actually specifies Manhattan distance to be used.

Nonetheless, to start off this puzzle I began trying to think of how I could identify which points had areas that were finite. After some thinking (initially with the assumption of using Euclidean distance), I first realized that any point with finite area must be in some way "enclosed" by a polygon formed by other points, and moreover, there has to be a triangle of other points that enclose it. So if for each point I iterated over all triples of other points, and if I could easily calculate if one point was within a triangle of other points, then I could determine which points have bounded areas. (Calculating this is not trivial, but I found a StackOverfloow post offering an easy calculation for if a point is within a 2D triangle, and willingly borrowed it for my educational use).

Later on, after realizing I had to use Manhattan distance, I had to reconsider this "finite area" identification subproblem, as the triangle-based solution no longer worked. One thing I noticed was that the diagrams formed by the points were very clearly Voronoi diagrams, as like the one shown in [this picture](https://commons.wikimedia.org/wiki/File:Manhattan_Voronoi_Diagram.svg). After enough studying of the picture, I concluded for a point to have finite area, then when considering the four diagonal quadrants surrounding the point (that is, visualized like the cartesian quadrants but at a 45 degree angle), there must be at least one point in each of the quadrants, assuming we label each point as the center of its four quadrants. So in a similar fashion as with the triangle idea, I iterated through all 4-combinations of points, and checked which would satisfy this four-quadrant constraint. (The exact checking method can be seen in the `point_in_diamond` function). Fortunately, as the input given only includes 50 points, 50^5 is just small enough number of iterations to be practical to handle all the combinations. Now, we have identified which of the 50 points will have finite areas.

Also after inspecting the input, I noticed that all 50 of the input points were within roughly a 400x400 grid space. This makes for about 160,000 individual "locations" on the grid, for which it would not be that expensive to calculate which input point each location is closest to. From here, it's just a matter of summing the number of locations associated with each input point, filtering out the input points with infinite space, and taking the maximum out of all of them to get the answer!

For the second star, the solution was easier as it just involve checking which points were within the constrained distance. Naturally, I just iterated over all of the locations, added the distance to each of the given points, and checked whether each was within the required constraint.

Overall the code I wrote takes a while to run - around 30 seconds to a minute it seems, so I reckon there are probably some much more efficient algorithms for the problem. But I really enjoyed this challenge as a whole!

Edit: Thanks to a kind suggestion of a friend, I was able to optimize the finite-area identification part of my algorithm from O(N^5) to O(N^2) (where N is the number of points). The trick is that we only actually need to identify the quadrant of each point once, to see if there is a point in the given quadrant. Thus, we do not actually have to know which four-tuple of points make up the quadrant. This allows me to calculate the solution to both parts within about 15 seconds now (with less than a second going towards the finite area calculation), so this is a substantial improvement.

In [3]:
# test = """1, 1
# 1, 6
# 8, 3
# 3, 4
# 5, 5
# 8, 9
# """

# X = test.split('\n')[:-1]

GRID_SIZE = 400

with open('input/input6.txt') as f:
    X = f.read().split('\n')[:-1]  # remove last entry corresponding to empty line

# points are stored as a list of coordinate tuples
points = list(map(lambda x: tuple(map(int, x.split(', '))), X))

def left_quad(x, test):
    return test[0] < x[0] and abs(test[1] - x[1]) < (x[0] - test[0])

def right_quad(x, test):
    return test[0] > x[0] and abs(test[1] - x[1]) < (test[0] - x[0])

def top_quad(x, test):
    return test[1] > x[1] and abs(test[0] - x[0]) < (test[1] - x[1])

def bottom_quad(x, test):
    return test[1] < x[1] and abs(test[0] - x[0]) < (x[1] - test[1])

# calculate which points have finite area
finite = [False] * len(points)
for i, point in enumerate(points):
    l = r = t = b = False
    for test_point in points:
        if left_quad(point, test_point):
            l = True
        elif right_quad(point, test_point):
            r = True
        elif top_quad(point, test_point):
            t = True
        elif bottom_quad(point, test_point):
            b = True
    if l and r and t and b:
        finite[i] = True

# this is about the half way point of the calculation,
# so its nice to get an update things are working!
print(finite)

def manhattan_dist(p0, p1):
    return abs(p1[0] - p0[0]) + abs(p1[1] - p0[1])

# calculate which point is closest for each location in the grid
grid = [[-1] * GRID_SIZE for _ in range(GRID_SIZE)]
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        shortest_index = 0
        shortest_dist = 2 * GRID_SIZE * GRID_SIZE
        for k in range(len(points)):
            dist = manhattan_dist(points[k], (i, j))
            if dist < shortest_dist:
                shortest_dist = dist
                shortest_index = k
            elif dist == shortest_dist:  # avoid ties
                shortest_index = -1
        grid[i][j] = shortest_index

# calculate which point (with finite area) has the most locations closest to it
flat_grid = [entry for row in grid for entry in row]
best_point = 0
best_count = 0
for i in range(len(points)):
    if not finite[i]:
        continue
    count = sum(map(lambda x: x == i, flat_grid))
    if count > best_count:
        best_count = count
        best_point = i

print(best_count)

# second star

TOTAL_DISTANCE = 10000

# store a grid indicating which positions are "centralized",
# i.e. within the total distance constraint
central = [[False] * GRID_SIZE for _ in range(GRID_SIZE)]
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        total = 0
        for k, point in enumerate(points):
            total += manhattan_dist(points[k], (i, j))
        if total < TOTAL_DISTANCE:
            central[i][j] = True

flat_central = [entry for row in central for entry in row]
print(sum(flat_central))

[True, True, False, True, True, False, False, False, True, False, False, False, True, False, False, True, True, False, True, True, False, True, True, True, True, True, True, True, True, False, False, False, True, True, True, False, False, False, False, False, True, False, True, True, True, True, True, False, True, True]
3722
44634


## [Day 7](https://adventofcode.com/2018/day/7): The Sum of Its Parts

The first star for this challenge was relatively straight forward, as I quickly identified that it just required a topological sort of the steps. For a minute or so I forgot how the algorithm worked (is it like a breadth first search starting from the root? or from the tail? or is it different?), but I've implemented it in Python before so I eventually recalled how it worked. The algorithm follows easily from the logic of the example they included.

The second star required just adding some additional components to handle the time-tracking logic separately. Essentially I just stored a dictionary of (letter, time-left) pairs to keep track of what is getting worked on.

In [31]:
# first star

with open('input/input7.txt') as f:
    X = f.readlines()

X = list(map(lambda x: (x[5], x[36]), X))

letters = set()
reqs = defaultdict(set)  # map from a required step to the set of steps that depend on it
rev_reqs = defaultdict(set)  # map from a step to the set of steps it requires
for req_step, step in X:
    reqs[req_step].add(step)
    rev_reqs[step].add(req_step)
    letters.add(step)
    letters.add(req_step)

ready = sorted([letter for letter in letters if len(rev_reqs[letter]) == 0])
sequence = []
while ready:
    step = heappop(ready)
    sequence.append(step)
    for succ in reqs[step]:
        rev_reqs[succ].remove(step)
        if len(rev_reqs[succ]) == 0:
            heappush(ready, succ)

print(''.join(sequence))

# second star

# re-initialize variables
sequence = []
rev_reqs = defaultdict(set)
for req_step, step in X:
    rev_reqs[step].add(req_step)
ready = sorted([letter for letter in letters if len(rev_reqs[letter]) == 0])

times = {letter: 61 + ord(letter) - ord('A') for letter in letters}
processing = dict()
time = 0
while ready or processing:
    # more steps that workers can begin work on
    while ready and len(processing) < 5:
        step = heappop(ready)
        processing[step] = times[step]
        
    # perform time-based updates
    keys = list(processing.keys())
    for key in keys:
        processing[key] -= 1
        if processing[key] == 0:
            del processing[key]
            sequence.append(key)
            for succ in reqs[key]:
                rev_reqs[succ].remove(key)
                if len(rev_reqs[succ]) == 0:
                    heappush(ready, succ)

    time += 1
    
print(''.join(sequence))
print(time)

BFKEGNOVATIHXYZRMCJDLSUPWQ
BFKVEGAOTNYIHXZRMCJLDSUPWQ
1020
